# Problem 2

We've been provided files of yearly game appearance statistics for every
player to have played in Major League Baseball between 1871 and 2014. The goal is to download these files and produce a list of triples of teams for which at least 50 players have played for all three teams. For example, Alex Rodriguez has played for the Mariners, Rangers, and Yankees, and thus he would count once for the Mariners/Rangers/Yankees triple. The time and space complexity of the algorithm should also be provided.

In the following, let $n_\text{files}$ represent the number of files, $n_\text{years}$ the number of years, $n_\text{players}$ the total number of players, and $n_\text{teams}$ the total number of teams.

## Download

The first step is to download the data.

In [2]:
import pandas as pd
import time

year_start, year_end = 1871, 2014

t0 = time.time()
df = pd.DataFrame()
for year in range(year_start, year_end):
    df_yr = pd.read_csv(
        'https://s3.amazonaws.com/dd-interview-data/data_scientist/baseball/appearances/'
        '{}/{}-0,000'.format(year, year),
        header=None, usecols=[1, 3], names=['team', 'player'])
    df = df.append(df_yr)
print('Time to download files:', time.time() - t0)

Time to download files: 33.86386299133301


In [3]:
df.head()

,team,player
0,RC1,hamra01
1,RC1,addybo01
2,CL1,bassjo01
3,RC1,birdge01
4,BS1,conefr01


Each row corresponds to a player on a team, regardless of year. That means it's likely there'll be rows with the same team and player. The other columns were not needed to solve the problem and hence were excluded to reduce actual space used.

In [4]:
print('Number of rows is', len(df))

Number of rows is 98146


The time complexity of the download step is $O(n_\text{files}) = O(n_\text{years})$. The worst-case space complexity is $O(n_\text{years}n_\text{players})$, but because the number of players in each file corresponds to a fraction of the total number of players, the average or amortized cost is expected to be much less.

## Algorithm

Now that we have the data, we need to find team triples in which at least 50 players have played for all three teams. This requires iterating over all triples and finding intersections between sets of players who've played for each team. Searching the DataFrame above by team to get a list of all players who've played for that team is $O(n)$, so we wouldn't want to do that for each team in each triple. Instead, we'll create an index on team so we can lookup by team key in $O(1)$.

In [5]:
df = df.set_index('team')

In [6]:
df.head()

,player
team,
RC1,hamra01
RC1,addybo01
CL1,bassjo01
RC1,birdge01
BS1,conefr01


Indexing requires $O(n_\text{years}n_\text{players})$ in the worst-case, but no added space complexity.

Next we'll get a list of unique teams. Because the teams have been hashed, this takes $O(1)$, with space complexity $O(n_\text{teams})$.

In [7]:
teams = df.index.unique()
print(teams)

Index(['RC1', 'CL1', 'BS1', 'WS3', 'CH1', 'TRO', 'FW1', 'PH1', 'NY2', 'BR2',
       ...
       'SEA', 'TOR', 'FLO', 'COL', 'ANA', 'ARI', 'MIL', 'TBA', 'WAS', 'MIA'],
      dtype='object', name='team', length=150)


In [8]:
print('Number of teams is', len(teams))

Number of teams is 150


So we can lookup in the DataFrame for all players who've played on a given team.

In [9]:
df.loc['RC1']['player']

team
RC1      hamra01
RC1     addybo01
RC1     birdge01
RC1     mackde01
RC1    ansonca01
RC1    barkeal01
RC1    fishech01
RC1    fulmech01
RC1    hastisc01
RC1    sagerpo01
RC1    stirega01
Name: player, dtype: object

Finding intersections between the Series for two teams is $O(n^2)$ (worst-case), since you potentially have to iterate through all elements in both arrays. We can hash players on teams as well using sets. This has the added effect of removing duplicates.

In [10]:
team_players = {}
for team in teams:
    team_players[team] = set(df.loc[team]['player'])

This has time and space complexities of $O(n_\text{teams}n_\text{players})$ (worst-case). Again, average or amortized costs will be significantly lower.

To get the answer, we have only to loop through all combinations of triples, look up the players for each team, and get the intersections. We'd like to cut down on the number of triples we have to consider and intersections we need to compute, so we'll create another hash table mapping pairs of teams to the set of their common players. That way, if those pairs don't have at least 50 players in common, they can't produce a triple and can be skipped.

In [13]:
from itertools import combinations

team_players_intersections = {}
for team_1, team_2 in combinations(teams, 2):
    team_players_intersections[(team_1, team_2)] = \
        team_players[team_1].intersection(team_players[team_2])

This has worst-case time and space complexities of $O(n_\text{teams}^2n_\text{players})$.

Check how many of these pairs have at least 50 players.

In [84]:
min_players_needed = 50
print(
    sum([len(v) >= min_players_needed for v in team_players_intersections.values()]))

372


That's two orders of magnitude smaller than the total number of double combinations (11,175).

Now simply look through all triples.

In [76]:
team_triples = []

t0 = time.time()
for team_1, team_2, team_3 in combinations(teams, 3):
    common_players = team_players_intersections[(team_1, team_2)]
    
    if len(common_players) < min_players_needed:
        continue
        
    if len(common_players.intersection(team_players[team_3])) >= min_players_needed:
        team_triples.append((team_1, team_2, team_3))

print('Done in', time.time() - t0)

Done in 0.21820402145385742


The time complexity is $O(n_\text{teams}^3n_\text{players})$ (worst-case). Space complexity is effectively $O(n_\text{players})$.

Here is the answer.

In [78]:
team_triples

[('CHN', 'PHI', 'CIN'),
 ('CHN', 'PHI', 'SLN'),
 ('CHN', 'PIT', 'CIN'),
 ('CHN', 'CIN', 'SLN'),
 ('PHI', 'PIT', 'CIN'),
 ('PHI', 'PIT', 'SLN'),
 ('PHI', 'CIN', 'SLN'),
 ('PIT', 'CIN', 'SLN'),
 ('CHA', 'CLE', 'BOS'),
 ('CHA', 'CLE', 'NYA')]

Verify.

In [89]:
for team_1, team_2, team_3 in team_triples:
    print(len(
        team_players[team_1].intersection(
        team_players[team_2]).intersection(
        team_players[team_3])))

54
68
53
74
54
50
80
63
52
50


## Complexity

$Time$. Adding up the above, the total time complexity of the algorithm (excluding download time) is effectively $O(n_\text{teams}^3n_\text{players})$ (worst-case). However, because the number of triples to consider was reduced a few orders of magnitude, and the number of players that have been on each team is much smaller than the total number of players, the actual cost is significantly lower, as shown. Specifically, if $n_\text{doubles}$ represents the number of pairs of teams with at least 50 players in common, then the complexity is more like $O(n_\text{doubles}n_\text{teams}n_\text{players})$.

$Space$. Adding up the above, the total space complexity is effectively $O(n_\text{teams}^2n_\text{players})$ (worst-case). Again, because the number of players on each team is much smaller than the total number, the actual cost is significantly lower.